In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model/1")

4/4 [==============================] - 0s 1ms/step - loss: 0.3906
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model/1/assets


In [2]:
# model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [3]:
!saved_model_cli show --dir my_model/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
    Option #

In [4]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   3706      0 --:--:-- --:--:-- --:--:--  3706
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Ign:5 https://developer.download.nvidia.cn/compute/cuda/repos/ubuntu1804/x86_64  InRelease33m
Ign:7 https://developer.download.nvidia.cn/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.cn/co

In [5]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tensorflow-model-server is already the newest version (2.2.0).
0 upgraded, 0 newly installed, 0 to remove and 64 not upgraded.


In [ ]:
# tensorflow_model_server   --rest_api_port=8501   --model_base_path="/tf/develop/tensorflow-101/experts/code_sample/chapter-7/my_model" --model_name=test

In [9]:
import json

test_input = np.random.random((10, 32)).tolist()

data = json.dumps({"signature_name": "serving_default", "instances": test_input})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... 936144825, 0.5185309751828353, 0.5023908954318431]]}


In [10]:
test_input

[[0.45436844163230994,
  0.05627256599276975,
  0.25591809270793164,
  0.029789544486036323,
  0.39569255510939494,
  0.7297838401133463,
  0.23742993456288586,
  0.8996363303931911,
  0.29382635031297666,
  0.025774003317361016,
  0.8040641769271422,
  0.06950509753452039,
  0.21925935817381603,
  0.18863782598441936,
  0.5805749762694452,
  0.6769084977837265,
  0.47579473171726316,
  0.5773005191777567,
  0.6531662164542267,
  0.25186803066891195,
  0.7913387473230628,
  0.4020227463536963,
  0.8157617668025108,
  0.8195900085845316,
  0.7934095541761969,
  0.9492209943467972,
  0.6250450461649494,
  0.3630605771474398,
  0.7624501040854188,
  0.924706956593921,
  0.6132740724949204,
  0.08585189124069181],
 [0.23367816864056,
  0.46511921932905587,
  0.8163823421097941,
  0.147512006689263,
  0.20003473397265437,
  0.9470153352191873,
  0.1524467815969739,
  0.26541536364364526,
  0.17331656425344633,
  0.8850017191250238,
  0.0043885991971420735,
  0.23702160657698235,
  0.3992559

In [11]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/test:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']
print(predictions)

[[-1.18497694], [0.0344912149], [-0.119354866], [0.71193248], [0.207333058], [0.717658937], [0.428872854], [0.178136736], [0.258845478], [0.210190967]]
